In [ ]:
%matplotlib inline

import sys
sys.path.append('../')

import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib

from src.utils import plot_list, read_images
from src.metrics import compute_ious, compute_eval_metric

VALIDATION_RESULTS_FILEPATH = '/mnt/ml-team/minerva/open-solutions/salt/kuba/experiments/sal_491_cv_796_lb_783/validation_results.pkl'

In [ ]:
meta_valid, y_true, y_pred = joblib.load(VALIDATION_RESULTS_FILEPATH)

In [ ]:
meta_valid.head()

In [ ]:
ious = [compute_ious(gt, pred)[0][0] for gt, pred in zip(y_true, y_pred)]
iouts = [compute_eval_metric(gt, pred) for gt, pred in zip(y_true, y_pred)]

raw_imgs = read_images(meta_valid['file_path_image'])
depths = meta_valid['z'].tolist()
results = list(zip(ious, iouts, depths, raw_imgs, y_true, y_pred))

# Score distributions

In [ ]:
sns.distplot(ious)

In [ ]:
sns.distplot(iouts)

# score by depth

In [ ]:
sns.regplot(iouts, depths, fit_reg=False,x_jitter=True)

# Predicted mask exploration

In [ ]:
def filter_results(results, iout_range):
    iout_min, iout_max = iout_range
    results_filtered = []
    for tup in results:
        if iout_min<=tup[1]<=iout_max:
            results_filtered.append(tup)
    return results_filtered

results_filtered = filter_results(results, iout_range=(0.0,0.2))
len(results_filtered)

In [ ]:
IMG_NR = 10

for iou, iout, z, img, gt, pred in results_filtered[:IMG_NR]:
    print('IOU {}, IOUT {}, depth {}'.format(iou, iout, z))
    plot_list(images=[img],labels=[gt, pred])